# Notebook dedicado a construcao da tabela a ser analisada

In [1]:
import json
import pandas
import collections
from core.MachineLearningOptions import MachineLearningOptions
from core.FeatureTransform import FeatureTransform
from core.DataExploration import DataExploration
from core.util import cleanDataColumns
from core.util import renameColumn
from hagana.ContatenateTables import ContatenateTables
from hagana.CalculateKpis import CalculateKpis

In [2]:
# se a pessoa ligou reclamando, se nao ligou
# se foram resolver
# se expandiram o posto de trabalho ou nao
# ficou mais caro ou nao

# PREPARACAO

In [3]:
from hagana.ContatenateTables import ContatenateTables
import pandas
fileInsp = 'hagana-inspecoes.xls'
fileRecis = 'hagana-recisoes.csv'
conT = ContatenateTables(fileInsp, fileRecis)
conT.generateRecisaoColStatus()
conT.dataInspecoes = conT.takeOnlyNumberFromGrade(conT.dataInspecoes)
conT.dataInspecoes = conT.transformTimeUnit(conT.dataInspecoes)
pandas.DataFrame(data=conT.getCodigosRecisoesNotFound()).to_csv('nao-encontrei-na-inspecao.csv',sep=';')
conT.dataInspecoes.to_csv('haganaTable.csv',sep=';')

In [4]:
#LOAD TABLE
data = pandas.read_csv('haganaTable.csv',delimiter=';',encoding='latin-1',dtype=str)

In [ ]:
targetColumn = list(data.columns).index('recisaoCol')
allOptions = {}
featT = FeatureTransform()

# TRANSFORMACOES DAS COLUNAS PARA O MODELO

In [ ]:
colName = 'indicaria'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'Sim':'S',
            'NÃ£o':'N'
        },
        'oneHotColumns': ['S', 'N'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
colName = 'sugestao'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'Sim':'S',
            'NÃ£o':'N'
        },
        'oneHotColumns': ['S', 'N'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
colName = 'area'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'False':'NULO',
            'DELTA 1':'DELTA',
            'DELTA 2':'DELTA',
            'DELTA 3':'DELTA',
            'ARCO 1':'ARCO',
            'ARCO 2':'ARCO',
            'Â¡GUIA 1':'AGUIA',
            'Â¡GUIA 2':'AGUIA',
            'RMS':'OUTROS',
            'RMSJ':'OUTROS',
            'RMC':'OUTROS',
            'EXTREMO OESTE':'OUTROS',
            'FLECHA':'OUTROS'
        },
        'oneHotColumns': ['PELICANO','DELTA', 'ARCO','AGUIA','OUTROS'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
colName = 'area'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'False':'NULO',
            'DELTA 1':'DELTA',
            'DELTA 2':'DELTA',
            'DELTA 3':'DELTA',
            'ARCO 1':'ARCO',
            'ARCO 2':'ARCO',
            'Â¡GUIA 1':'AGUIA',
            'Â¡GUIA 2':'AGUIA',
            'RMS':'OUTROS',
            'RMSJ':'OUTROS',
            'RMC':'OUTROS',
            'EXTREMO OESTE':'OUTROS',
            'FLECHA':'OUTROS'
        },
        'oneHotColumns': ['PELICANO','DELTA', 'ARCO','AGUIA','OUTROS'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
colName = 'falta'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'PerÃ\x8codo Noturno':'RECLAMOU',
            'PerÃ\x8codo Diurno':'RECLAMOU',
            'Ambos os PerÃ\x8codo':'RECLAMOU',
        },
        'oneHotColumns': ['RECLAMOU'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
colName = 'cargo'
allOptions[colName] = {
    'type': 'cat',
    'transforms': {
        'convertCategories':{
            'Gerente Predial':'Sindico',
            'Sindico Profissional':'Sindico',
            'SÃ\xadndico ':'Sindico',
            'Administrador':'Sindico',
            'Zelador':'Zelador',
            'Gerente Administrativo':'Gerente',
            'Gerente de SeguranÃ§a':'Gerente',
            'Diretor de SeguranÃ§a':'Gerente',
        },
        'oneHotColumns': ['Sindico','Gerente','Zelador'],
        'fillna':'NULO'
    }
}
featT.transform(data, colName,allOptions)

de = DataExploration(data)
indexI = list(data.columns).index(colName)
de._categoryPercentage(indexI,targetColumn)

In [ ]:
with open('Hagana-01.json', 'w') as outfile:
    json.dump(allOptions, outfile)

In [5]:
# CARREGANDO DOS DADOS PARA O MODELO
data = pandas.read_csv('haganaTable.csv',delimiter=';',encoding='latin-1',dtype=str)
with open('Hagana-01.json') as f:
    allOptions = json.load(f)
featT = FeatureTransform()
for colName in allOptions:
    featT.transform(data, colName,allOptions) # dropa as colunas transformadas com One Hot Encoder
columnNames = featT.getAllColumnsNames(allOptions)
columnNames += ['supervisao','grupo', 'diurna', 'noturna', 'equipamentos']
columnNames += ['data']

# GERACAO DAS KPIS

In [6]:
calculateKpis = CalculateKpis(data)

In [7]:
recisaoCodes = conT.getCodigosRecisao()
meanOfStatsRecisao = calculateKpis.kpiOfCodes(recisaoCodes)

In [8]:
allCodes = list(collections.Counter(data.loc[:,'codigo']).keys())
semRecisao = []
for code in allCodes:
    if code not in recisaoCodes:
        semRecisao.append(code)
meanOfStatsSemRecis = calculateKpis.kpiOfCodes(semRecisao)

In [ ]:
calculateKpis.plotAllGraphics(meanOfStatsRecisao,meanOfStatsSemRecis)

In [ ]:
# dicionario = [cara e as possiveis caracteristicas]

In [ ]:
#

In [ ]:
update(pessoa & coisa)